In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from common import calculate_centroid_geo, haversine_np

In [ ]:
df = pd.read_csv("data/mappers_processed.csv")

In [ ]:
# Limit to SF Bay
lat_bounds = [37, 38]
long_bounds = [-124, -122]

df = df[df["target_lat"] > min(lat_bounds)]
df = df[df["target_lat"] < max(lat_bounds)]
df = df[df["target_long"] > min(long_bounds)]
df = df[df["target_long"] < max(long_bounds)]

In [ ]:
subset = df.sample(n=min(len(df), 5000))
for i in range(1,4):
    plt.scatter(subset[f"hotspot{i}_long"], subset[f"hotspot{i}_lat"], s=2, alpha=0.8, label=f"hotspot{i}")
plt.scatter(subset["target_long"], subset["target_lat"], s=2, label="label")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Random sampling of data points")
plt.legend()

In [ ]:
# Limit the number of samples
max_samples = 10000
if df.shape[0] > max_samples:
    print(f"Limiting # samples from {df.shape[0]} to {max_samples}")
    df = df.sample(n=max_samples)

In [ ]:
data = np.concatenate([
    df[[f"{prefix}_lat", f"{prefix}_long"]].to_numpy() for prefix in ["target", "hotspot1", "hotspot2", "hotspot3"]
])
print(data.shape)

In [ ]:
def k_means(data, k:int=50, num_iter:int=50, seed:int=42, trace: bool=False):
    np.random.seed(seed)

    n, d = data.shape
    data_lla = np.stack([data[:,0], data[:,1], np.zeros((n))])
    
    # Initialize centroids as random selection of data points
    centroids = data[np.random.choice(n, k, replace=False)] # k, d 
    diff = np.zeros((n,k))

    for _ in tqdm(range(num_iter), disable=not trace):
        # Assign centroids based on minimum haversine distance
        for i in range(k):
            diff[:, i] = haversine_np(data, centroids[np.newaxis, i,:])
        labels = diff.argmin(axis=1) # n,

        # Update the centroids to be the projected centroid of the members of each cluster
        for i in range(k):

            members = data_lla[np.argwhere(labels==i), :]
            if members.shape[0] == 0:
                # empty cluster... don't update
                continue
            centroids[i] = calculate_centroid_geo(lat_long_alt=members[:,:])[:,:2]
        
    return centroids

In [ ]:
# Cluster
centroids = k_means(data, k=2000, trace=True)

In [ ]:
# Visualize clusters
subset = data[np.random.choice(data.shape[0], min(10000, data.shape[0]), replace=False), :]
plt.scatter(subset[:,1], subset[:,0], alpha=0.1, label="data")
plt.scatter(centroids[:,1], centroids[:,0], marker="*", color="k", label="cluster centroid")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.title(f"K-Means Clusters k={centroids.shape[0]}")
plt.show()